In [ ]:
!pip install bs4
!pip install requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import warnings
from ipywidgets import interactive


In [ ]:
main_url = 'https://genshin-info.ru/wiki/'

In [ ]:
html = requests.get(main_url + 'personazhi/').text
soup = BeautifulSoup(html, 'html')
itemcards = soup.find_all(lambda tag: tag.name == 'a' and tag.get('class') == ['itemcard'])

In [ ]:
def get_character_data(info, idx, name):
    data = info.find_all('span', {'class':'characterPromo__propV'})
    rarity = len(data[0].find_all('i', {'class':'fa fa-star'}))

    rating = info.find('div', {'title':'Общий рейтинг'}).text.strip() if info.find('div', {'title':'Общий рейтинг'}) else None

    gods_eye = data[1].text.strip() if len(data) > 1 else None
    weapon = data[2].text.strip() if len(data) > 2 else None

    return {
        'index_character': idx,
        'name': name,
        'rarity': rarity,
        'rating': rating,
        'gods_eye': gods_eye,
        'weapon': weapon
    }

In [ ]:
items = []
characters = []
world = "Genshin Impact"
group = "Мобильные Игры"
idx = 0
for item in itemcards:
  name = item.get('title')

  character_url = main_url + 'personazhi/' + list(filter(None, item.get('href').split('/')))[-1]
  print(character_url)
  character_html = requests.get(character_url).text
  info = BeautifulSoup(character_html, 'html')

  characters.append(get_character_data(info, idx, name))

  items.append({
              'index_character': idx,
              'group': group,
              'world': world,
              'name': name,
              })
  idx += 1

In [ ]:
df = pd.DataFrame(characters)
df.to_excel('genshin.xlsx', index = False)
df

In [ ]:
df2 = pd.DataFrame(characters)
df2.to_excel('genshin_Characters.xlsx', index = False)
df2

In [ ]:
file_path = "genshin_Characters.xlsx"
df = pd.read_excel(file_path)

In [ ]:
def select_rating(ch1="Нилу", ch2="Нилу"):
    warnings.filterwarnings('ignore')

    character1 = df[df['name'] == ch1]
    character2 = df[df['name'] == ch2]

    categories = [ch1, ch2]
    values = [int(character1['rarity']), int(character2['rarity'])]

    plt.bar(categories, values)
    plt.show()

In [ ]:
w = interactive(select_rating, ch1=widgets.Dropdown(options=df['name'].unique(), description='Hero1:'), ch2=widgets.Dropdown(options=df['name'].unique(), description='Hero2:'))
display(w)

In [ ]:
def select_rating(gods_eye = "Пиро"):
  warnings.filterwarnings('ignore')
  fig, axs = plt.subplots(1, 2, figsize=(12, 6))

  characters_with_gods_eye = df[df['gods_eye'] == gods_eye]

  rating_mapping = {'D': 1, 'C': 2, 'B': 3, 'A': 4, 'S': 5, 'S+': 6}
  characters_with_gods_eye['rating_numeric'] = characters_with_gods_eye['rating'].map(rating_mapping)
  characters_with_gods_eye = characters_with_gods_eye.sort_values(by='rating_numeric')

  color_mapping = {
      "Анемо": "#71c2a7",
      "Электро": "#b38dc1",
      "Гидро ": "#5fc1f1",
      "Крио": "#a4d6e3",
      "Пиро": "#ea7838",
      "Гео": "#f2b723",
      "Дендро": "#9cc928",
  }
  color = color_mapping.get(gods_eye, '#ea7838')

  axs[0].barh(characters_with_gods_eye['name'], characters_with_gods_eye['rating_numeric'], color=color)
  axs[0].set_title('Рейтинг персонажей')

  gods_eye = df['gods_eye'].value_counts().sort_values()
  gods_eye.plot(kind='bar', ax=axs[1], color=color)
  for i, count in enumerate(gods_eye):
      axs[1].text(i, count+0.1, str(count), va='bottom', ha='center')

  axs[1].set_xlabel('Глаз бога')
  axs[1].set_ylabel('Количество персонажей')
  axs[1].set_title('Частота встречаемости глаза бога')
  axs[1].set_yticks([])

  plt.show()

In [ ]:
w = interactive(select_rating,  gods_eye = widgets.Dropdown(options=df['gods_eye'].unique(), description='Глаз бога:'))
display(w)